In [21]:
import doctest
import itertools as it
from typing import List, Tuple

import bs4
import urllib

def get_title(day):
    f = urllib.request.urlopen(f"https://adventofcode.com/2019/day/{day}")
    soup = bs4.BeautifulSoup(f.read())
    print(soup.h2.text.replace("-","").strip())

## Day 1: The Tyranny of the Rocket Equation

In [2]:
def day01_parse(file_):
    return [int(line) for line in open(file_).readlines()]
    

def day01(mass: int) -> int:
    """
    >>> day01(12)
    2
    >>> day01(14)
    2
    >>> day01(1969)
    654
    >>> day01(100756)
    33583
    """
    return mass // 3 - 2


In [3]:
inputs = day01_parse("./2019/day01.txt")
output = sum(day01(x) for x in inputs)
output

3286680

In [4]:
def day01_mod(mass: int) -> int:
    """
    >>> day01_mod(14)
    2
    >>> day01_mod(1969)
    966
    >>> day01_mod(100756)
    50346
    """
    res = []
    while True:
        mass = day01(mass)
        if mass > 0:
            res.append(mass)
        else:
            break
    return sum(res)

In [5]:
doctest.testmod()

TestResults(failed=0, attempted=7)

In [6]:
inputs = day01_parse("./2019/day01.txt")
output = sum(day01_mod(x) for x in inputs)
output

4927158

## Day 2: 1202 Program Alarm

In [7]:
def day02_parse(file_):
    return [int(x) for x in open(file_).read().strip().split(",")]
   
    
def _day02_op1(xs: List[int], loc: int) -> List[int]:
    xs[xs[loc + 3]] = xs[xs[loc + 1]] + xs[xs[loc + 2]]
    return xs, loc + 4


def _day02_op2(xs: List[int], loc: int) -> List[int]:
    xs[xs[loc + 3]] = xs[xs[loc + 1]] * xs[xs[loc + 2]]
    return xs, loc + 4


def day02(state: List[int]) -> int:
    """
    >>> day02([1,9,10,3,2,3,11,0,99,30,40,50])
    3500
    >>> day02([1,0,0,0,99])
    2
    >>> day02([2,4,4,5,99,0])
    2
    >>> day02([1,1,1,4,99,5,6,0,99])
    30
    """
    loc = 0
    while state[loc] != 99:
        if state[loc] == 1:
            state, loc = _day02_op1(state, loc)
        elif state[loc] == 2:
            state, loc = _day02_op2(state, loc)
    return state[0]

In [8]:
doctest.testmod()

TestResults(failed=0, attempted=11)

In [9]:
inputs = day02_parse("./2019/day02.txt")
inputs[1] = 12
inputs[2] = 2
day02(inputs[:])

9706670

In [10]:
def day02_mod(xs: List[int], target: int) -> int:
    size = len(xs)
    for noun in range(size):
        for verb in range(size):
            ys = xs[:]
            ys[1] = noun
            ys[2] = verb
            if day02(ys) == target:
                return 100 * noun + verb

In [11]:
day02_mod(inputs, 19690720)

2552

## Day 3: Crossed Wires

### Part 1

In [13]:
def _day03_exec_command(cmd, state):
    dir_, steps = cmd[0], int(cmd[1:])
    coord, record = state
    x, y = coord
    if dir_ == "R":
        dx, dy = 1, 0
    elif dir_ == "L":
        dx, dy = -1, 0
    elif dir_ == "U":
        dx, dy = 0, 1
    else:
        assert dir_ == "D"
        dx, dy = 0, -1
        
    for i in range(1, steps + 1):
        record.add((x + dx * i, y + dy * i))
                    
    coord = (x + dx * steps, y + dy * steps)
    return coord, record


def day03_parse(file_):
    ss = open(file_).readlines()
    res = [s.strip() for s in ss]
    return res


def _day03_process(commands):
    commands = commands.split(",")
    state = (0, 0), set()
    for cmd in commands:
        state = _day03_exec_command(cmd, state)

    _, record = state
    return record


def day03(commands1, commands2):
    """
    >>> day03("R8,U5,L5,D3", "U7,R6,D4,L4")
    6
    
    >>> day03("R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83")
    159
    
    >>> day03("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
    135
    
    """
    rec1 = _day03_process(commands1)
    rec2 = _day03_process(commands2)
    crossings = rec1 & rec2
    nearest = min(sum(map(abs, point)) for point in crossings)
    return nearest

In [14]:
doctest.testmod()

TestResults(failed=0, attempted=14)

In [15]:
command1, commands2 = day03_parse("./2019/day03.txt")
day03(command1, commands2)

651

### Part 2

There is ambiguity of step count when a trajectory crosses by itself. Assume that such self-crossing points do not appear as crossing points between trajectories.

In [16]:
def _day03_exec_command_mod(cmd, state):
    coord, record = state
    steps = record[coord]
    dir_, stride = cmd[0], int(cmd[1:])
    x, y = coord
    if dir_ == "R":
        dx, dy = 1, 0
    elif dir_ == "L":
        dx, dy = -1, 0
    elif dir_ == "U":
        dx, dy = 0, 1
    else:
        assert dir_ == "D"
        dx, dy = 0, -1
        
    for i in range(1, stride + 1):
        coord = (x + dx * i, y + dy * i)
#         if coord in record:
#             print("Already visited!")
        record[coord] = steps + i
                    
    coord = (x + dx * stride, y + dy * stride)
    return coord, record


def _day03_process_mod(commands):
    commands = commands.split(",")
    state = (0, 0), {(0, 0): 0}
    for cmd in commands:
        state = _day03_exec_command_mod(cmd, state)
    _, record = state
    return record


def day03_mod(commands1, commands2):
    """
    >>> day03_mod("R8,U5,L5,D3", "U7,R6,D4,L4")
    30
    
    >>> day03_mod("R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83")
    610
    
    >>> day03_mod("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
    410
    """
    rec1 = _day03_process_mod(commands1)
    rec2 = _day03_process_mod(commands2)
    crossings = (set(rec1.keys()) & set(rec2.keys())) - {(0, 0)}
    nearest = min(rec1[point] + rec2[point] for point in crossings)
    return nearest

In [17]:
doctest.testmod()

TestResults(failed=0, attempted=17)

In [18]:
command1, commands2 = day03_parse("./2019/day03.txt")
day03_mod(command1, commands2)

7534